# Setting up

In [232]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from EDA_Functions import sort_coo, extract_topn_from_vector
import pymorphy2
from nltk.stem import WordNetLemmatizer
import re
import numpy as np
from tqdm import tqdm

In [19]:
# reading data to variable
data = pd.read_csv('Data/With_keywords.csv', index_col=0)

In [20]:
# reading data to variable
data_2 = pd.read_csv('Data/itmo_23.04.2020_15:31.csv', index_col=0)

# dropping unvaluable columns
data_2 = data_2.drop(['all_keywords'], axis=1)

# initializing new column at data 2 data set with many words keywords
data_2['many_words_keywords'] = data['many_words_keywords']

# reading data to variable 
cleaned_data = pd.read_csv('Data/Cleaned_21_05.csv', index_col=0)

# initializing new column with clean description
data_2['cleaned_description'] = cleaned_data['Чистое содержание']

In [21]:
del data, cleaned_data

In [26]:
data_2 = data_2.fillna(' ')

In [22]:
data_2.head()

,SUBFIELDCODE,SUBFIELDNAME,DEGREE,SUBJECT,SEMESTER,START_YEAR,URL,Название курса,Содержание курса,Дата сет,Ключевые слова,key_words,many_words_keywords,cleaned_description
0,09.03.01,"Вычислительные машины, комплексы, системы и сети",Академический бакалавр,Математика,1,2014,https://www.hse.ru/edu/courses/292700221,Математика,Элементы линейной алгебрыТема 1.1. линейные пр...,hse,,"вероятность, распределение, экстремум, сложной...","['система линейный уравнение', 'вектор линейны...",элемент линейный алгебрытема линейный простран...
1,09.03.01,"Вычислительные машины, комплексы, системы и сети",Академический бакалавр,Математика,2,2014,https://www.hse.ru/edu/courses/292700221,Математика,Элементы линейной алгебрыТема 1.1. линейные пр...,hse,,"вероятность, распределение, экстремум, сложной...","['система линейный уравнение', 'вектор линейны...",элемент линейный алгебрытема линейный простран...
2,09.03.01,"Вычислительные машины, комплексы, системы и сети",Академический бакалавр,Физика,1,2014,https://www.hse.ru/edu/courses/292684119,Физика,Тема 1. Механика. Механические колебания и вол...,hse,,"импульс, поток, напряжённость, вектор, электро...","['волныкинематика колебание механический', 'ко...",механик механический колебание волныкинематика...
3,09.03.01,"Вычислительные машины, комплексы, системы и сети",Академический бакалавр,Физика,2,2014,https://www.hse.ru/edu/courses/292684119,Физика,Тема 1. Механика. Механические колебания и вол...,hse,,"импульс, поток, напряжённость, вектор, электро...","['волныкинематика колебание механический', 'ко...",механик механический колебание волныкинематика...
4,09.03.01,"Вычислительные машины, комплексы, системы и сети",Академический бакалавр,Основы права,1,2014,https://online.edu.ru/public/course.xhtml?face...,Основы права,Что такое право? Российское право на правовой...,itmo,,"семья, правонарушение, правового регулирования...","['гражданский собственность право', 'правовой ...",право российский право правовой карта мир взаи...


# Extracting keywords and their simularities using tf-idf

In [35]:
def tf_ifd_keywords(data):
    # creating Count vectorizer instance
    vectorizer = CountVectorizer(max_df=0.8, max_features=10000, ngram_range=(1, 3))

    # getting bag of words
    bow = vectorizer.fit_transform(data)
    
    # creating TF-IDF transformer instance
    tf_idf_transformer = TfidfTransformer(smooth_idf=True, use_idf=True)

    # fit bag of words from previous stem
    tf_idf_transformer.fit(bow)

    # getting feature names
    feature_names = vectorizer.get_feature_names()
    
    # initializing list for TF-IDF key words
    key_words_tf_idf = []

    # getting key words for each course
    for doc in data.values:
        # transforming doc vector to TF-IDF
        tf_idf_vector = tf_idf_transformer.transform(vectorizer.transform([doc]))

        # sorting sparse matrix
        sorted_items = sort_coo(tf_idf_vector.tocoo())

        # extracting to 15 keywords
        keywords = extract_topn_from_vector(feature_names, sorted_items, 15)

        # filling key words list
        key_words_tf_idf.append(keywords)
        
    return key_words_tf_idf

In [36]:
data_2['tf_idf_keywords'] = tf_ifd_keywords(data_2['cleaned_description'])

In [37]:
data_2.head()

,SUBFIELDCODE,SUBFIELDNAME,DEGREE,SUBJECT,SEMESTER,START_YEAR,URL,Название курса,Содержание курса,Дата сет,Ключевые слова,key_words,many_words_keywords,cleaned_description,tf_idf_keywords
0,09.03.01,"Вычислительные машины, комплексы, системы и сети",Академический бакалавр,Математика,1,2014,https://www.hse.ru/edu/courses/292700221,Математика,Элементы линейной алгебрыТема 1.1. линейные пр...,hse,,"вероятность, распределение, экстремум, сложной...","['система линейный уравнение', 'вектор линейны...",элемент линейный алгебрытема линейный простран...,"{'функция': 0.325, 'случайный': 0.233, 'переме..."
1,09.03.01,"Вычислительные машины, комплексы, системы и сети",Академический бакалавр,Математика,2,2014,https://www.hse.ru/edu/courses/292700221,Математика,Элементы линейной алгебрыТема 1.1. линейные пр...,hse,,"вероятность, распределение, экстремум, сложной...","['система линейный уравнение', 'вектор линейны...",элемент линейный алгебрытема линейный простран...,"{'функция': 0.325, 'случайный': 0.233, 'переме..."
2,09.03.01,"Вычислительные машины, комплексы, системы и сети",Академический бакалавр,Физика,1,2014,https://www.hse.ru/edu/courses/292684119,Физика,Тема 1. Механика. Механические колебания и вол...,hse,,"импульс, поток, напряжённость, вектор, электро...","['волныкинематика колебание механический', 'ко...",механик механический колебание волныкинематика...,"{'колебание': 0.285, 'энергия': 0.256, 'вектор..."
3,09.03.01,"Вычислительные машины, комплексы, системы и сети",Академический бакалавр,Физика,2,2014,https://www.hse.ru/edu/courses/292684119,Физика,Тема 1. Механика. Механические колебания и вол...,hse,,"импульс, поток, напряжённость, вектор, электро...","['волныкинематика колебание механический', 'ко...",механик механический колебание волныкинематика...,"{'колебание': 0.285, 'энергия': 0.256, 'вектор..."
4,09.03.01,"Вычислительные машины, комплексы, системы и сети",Академический бакалавр,Основы права,1,2014,https://online.edu.ru/public/course.xhtml?face...,Основы права,Что такое право? Российское право на правовой...,itmo,,"семья, правонарушение, правового регулирования...","['гражданский собственность право', 'правовой ...",право российский право правовой карта мир взаи...,"{'право': 0.661, 'ответственность': 0.268, 'пр..."


# Counting similarities of all keywords

In [40]:
ru_lemmatizer = pymorphy2.analyzer.MorphAnalyzer()
en_lemmatizer = WordNetLemmatizer()

In [66]:
new_key_phrases = []

for course in data_2.values:
    # getting key phrases
    key_phrases = course[-4].split(', ')
    
    course_key_phrases = []
    
    if len(key_phrases) != 0:   
        for key_phrase in key_phrases:
            key_words = key_phrase.split()
            key_words_lemmatized_list = []

            for word in key_words:
                if re.search(r'[А-Яа-я]+', word):
                    key_words_lemmatized_list.append(ru_lemmatizer.parse(word)[0].normal_form)
                else:
                    key_words_lemmatized_list.append(en_lemmatizer.lemmatize(word))

            key_words_lemmatized = ' '.join(key_words_lemmatized_list)

            phrase_similarity = 0

            for word, sim in course[-1].items():
                if re.search(word, key_words_lemmatized):
                    phrase_similarity += sim

            if len(key_words) != 0:
                if phrase_similarity / len(key_words) > 0.04:
                    course_key_phrases.append(key_phrase)
                
        new_key_phrases.append(', '.join(course_key_phrases))
    else:
        new_key_phrases.append(' ')

In [67]:
data_2['new_key_words'] = new_key_phrases

In [68]:
data_2.head()

,SUBFIELDCODE,SUBFIELDNAME,DEGREE,SUBJECT,SEMESTER,START_YEAR,URL,Название курса,Содержание курса,Дата сет,Ключевые слова,key_words,many_words_keywords,cleaned_description,tf_idf_keywords,new_key_words
0,09.03.01,"Вычислительные машины, комплексы, системы и сети",Академический бакалавр,Математика,1,2014,https://www.hse.ru/edu/courses/292700221,Математика,Элементы линейной алгебрыТема 1.1. линейные пр...,hse,,"вероятность, распределение, экстремум, сложной...","['система линейный уравнение', 'вектор линейны...",элемент линейный алгебрытема линейный простран...,"{'функция': 0.325, 'случайный': 0.233, 'переме...","вероятность, распределение, экстремум, сложной..."
1,09.03.01,"Вычислительные машины, комплексы, системы и сети",Академический бакалавр,Математика,2,2014,https://www.hse.ru/edu/courses/292700221,Математика,Элементы линейной алгебрыТема 1.1. линейные пр...,hse,,"вероятность, распределение, экстремум, сложной...","['система линейный уравнение', 'вектор линейны...",элемент линейный алгебрытема линейный простран...,"{'функция': 0.325, 'случайный': 0.233, 'переме...","вероятность, распределение, экстремум, сложной..."
2,09.03.01,"Вычислительные машины, комплексы, системы и сети",Академический бакалавр,Физика,1,2014,https://www.hse.ru/edu/courses/292684119,Физика,Тема 1. Механика. Механические колебания и вол...,hse,,"импульс, поток, напряжённость, вектор, электро...","['волныкинематика колебание механический', 'ко...",механик механический колебание волныкинематика...,"{'колебание': 0.285, 'энергия': 0.256, 'вектор...","импульс, поток, напряжённость, вектор, электро..."
3,09.03.01,"Вычислительные машины, комплексы, системы и сети",Академический бакалавр,Физика,2,2014,https://www.hse.ru/edu/courses/292684119,Физика,Тема 1. Механика. Механические колебания и вол...,hse,,"импульс, поток, напряжённость, вектор, электро...","['волныкинематика колебание механический', 'ко...",механик механический колебание волныкинематика...,"{'колебание': 0.285, 'энергия': 0.256, 'вектор...","импульс, поток, напряжённость, вектор, электро..."
4,09.03.01,"Вычислительные машины, комплексы, системы и сети",Академический бакалавр,Основы права,1,2014,https://online.edu.ru/public/course.xhtml?face...,Основы права,Что такое право? Российское право на правовой...,itmo,,"семья, правонарушение, правового регулирования...","['гражданский собственность право', 'правовой ...",право российский право правовой карта мир взаи...,"{'право': 0.661, 'ответственность': 0.268, 'пр...","семья, правонарушение, правового регулирования..."


In [69]:
data_2.iloc[0]

SUBFIELDCODE                                                    09.03.01
SUBFIELDNAME            Вычислительные машины, комплексы, системы и сети
DEGREE                                            Академический бакалавр
SUBJECT                                                       Математика
SEMESTER                                                               1
START_YEAR                                                          2014
URL                             https://www.hse.ru/edu/courses/292700221
Название курса                                                Математика
Содержание курса       Элементы линейной алгебрыТема 1.1. линейные пр...
Дата сет                                                             hse
Ключевые слова                                                          
key_words              вероятность, распределение, экстремум, сложной...
many_words_keywords    ['система линейный уравнение', 'вектор линейны...
cleaned_description    элемент линейный алгебрытема

In [71]:
data_2.iloc[0]['new_key_words']

'вероятность, распределение, экстремум, сложной функции, переменных функции, линейных пространств векторы, линейных алгебраических уравнений, линейный коэффициент корреляции, случайная величина'

In [72]:
data_2.iloc[10]

SUBFIELDCODE                                                    09.03.01
SUBFIELDNAME            Вычислительные машины, комплексы, системы и сети
DEGREE                                            Академический бакалавр
SUBJECT                                              Физическая культура
SEMESTER                                                               2
START_YEAR                                                          2014
URL                    https://online.edu.ru/public/course.xhtml?face...
Название курса                                      Физическая культура 
Содержание курса        Тема 1. Общая концепция теории и методики физ...
Дата сет                                                            itmo
Ключевые слова         здоровый образ жизни студента, спорт, физиолог...
key_words              физическими упражнениями, физической культуры,...
many_words_keywords    ['упражнение физический', 'человек биохимическ...
cleaned_description    физический культура физическ

In [76]:
data_2.iloc[10]['Содержание курса']

' Тема 1. Общая концепция теории и методики физической культуры Лекция 1. Введение в теорию физической культуры Лекция 2. Физическое воспитание, как центральная категория физической культуры Тема 2. Социально-биологические основы физической культуры, диагностика и здоровый образ жизни Лекция 3. Психофизиологические аспекты адаптации человека Лекция 4. Физиологические и биохимические основы физической культуры Тема 3. Самостоятельная двигательная активность Лекция 5. Методические основы самостоятельной физической тренировки Лекция 6. Самоконтроль занимающихся физическими упражнениями Лекция 7. Здоровый образ жизни студента Тема 4. Средства, формы и методы физической культуры в профессиональной деятельности человека Лекция 8. Основы профессионально-прикладной физической подготовки Тема 5. Спорт - интегративный фактор физической подготовки Лекция 9. Структура и социальные функции спорта '

In [73]:
data_2.iloc[10]['new_key_words']

'физическими упражнениями, физической культуры, здоровый образ жизни, центральная физической культуры, прикладной физической спорт, физическое воспитание'

In [74]:
data_2.iloc[23]

SUBFIELDCODE                                                    09.03.02
SUBFIELDNAME           Автоматизация и управление в образовательных с...
DEGREE                                            Академический бакалавр
SUBJECT                                                 Иностранный язык
SEMESTER                                                               2
START_YEAR                                                          2014
URL                             https://www.hse.ru/edu/courses/301401516
Название курса                                          Иностранный язык
Содержание курса       Личность. Описание внешности, физических харак...
Дата сет                                                             hse
Ключевые слова                                                          
key_words              грамматика, коммуникативные humans living, язы...
many_words_keywords    ['внешность личность', 'living коммуникативный...
cleaned_description    личность внешность физически

In [235]:
handed_stop_words = ['тема', 'модуль', 'раздел', 'темы']

In [152]:
change = {'документовструктура' : 'структура документов', 'научных документовструктура текста' : 'структура научных документов',
         'systemsinformation': 'systems information', 'statisticspopulation' : 'statistics population', 'цепейэлектрическая' : 'электрические цепи',
         'processproject': 'process project', 'management1' : 'management', 'managementinvestment': 'management investment',
         'ffff' : '', 'r2классификация' : 'r2 классификация', 'деятельностипонятие' : 'понятие деятельности',
          'деятельностивиды' : 'виды деятельности', 'introduction1' : '', 'стартапаисточник' : 'стартап', 'лидерствовид' : 'лидерство',
          'аналитикаструктура' : 'структура аналитики', 'моделимультиферроики' : 'модели мультиферроики',
          'технологийблок' : 'блок технологий', 'клиентаот' : 'клиент аот', 'managementequivalent' :'management equivalent',
          'интервьюфокус' : 'интервью', 'информацииконтент' : 'информационный контент', 'исследованияпонятие' : '',
          'исследованиитеории' : '', 'студентоввыступление': 'выступление студентов', 'тматрица' : 'матрица',
          'tariffstrategic' : 'tariff strategic', 'debateenvironment' : 'debate environment', 'ситуациейнаправления' : '',
          'structureidealism' : 'structure idealism','processmarkov' : 'Markov process', 'processpoisson' : 'Poisson process',
          'chaingaussian' : 'Gaussian chain', 'проектасоздание' : 'создание проекта', 'кубитовклассическая' : 'кубитов классическая',
          'presentationsguest' : 'presentations guest', 'сетейопределение' : 'определение сетей', 'zabbixустановка' : 'zabbix установка',
          'деятельностьпонятие' : 'понятие деятельности', 'заданиякомпозиция' : 'композиция', 'буклеттестирование' : '',
          'интерфейсастуденты': 'интерфейс', 'закладоккомпьютерные' : 'компьютерные закладки', 'поставокоптимизация' : 'оптимизация поставок',
          'гмувведение' : 'введение в гму', 'стэкнаследование' : 'стэк, наследование', 'мальтусэкономическая' : 'экономические взгляды Мальтуса',
          'discourselexicone' : 'lexicone', 'morphologysyntax' : 'morphology syntax', 'чтениеграмматика' : 'чтение, грамматика',
          'managementproject': 'management project', 'businessproduct' : 'business product', 'businessbusiness' : '',
          'проектахбизнес' : 'бизнес проекты', 'innovationprotection' : 'innovation protection', 'поставоктехнология' : 'технология поставок',
          'отношенияфункция' : 'функция отношения', 'эйконаланелинейные' : 'уравнение эйконала', 'интеграловосновные' : 'основные интегралы',
          'частицамидвижение': 'движение частиц', 'корпорациисистема' : 'система корпораций', 'этапасодержание': '',
          'инженериистратегия' : 'стратегия инженерии', 'компанияпринципы' : 'принципы компании','обеспечениясоздание' : 'создание обеспечения',
          'материаловедение' : 'ведение материалов', 'системераздел' : '', 'валентностипонятие' : 'понятие валентности',
          'pythonалгоритмы' : 'python алгоритмы', 'интернетработа' : 'работа в интернете', 'данныеконцепция' : 'концепция данных',
          '2вязкая' : 'вязкая', 'алгоритмывведение' : 'введение в алгоритмы', 'techniquesnaive' : 'naive techniques', 
          'selectionregression' : 'selection regression', 'industriestopic' : 'industries topic', 'stylesequilibrium' : 'styles equilibrium',
          'правапредмет' : 'предмет права', 'marketinvestment' : 'market investment', 'анализбюджетирование' : 'анализ бюджетирования',
          'тестовпрактические' : '', 'мышлениимышление': 'мышление', 'восприятииосновные' : 'основные восприятия',
          'перспективуоценка' : 'оценка перспективы', 'системыосновные' : '', 'типологияотношение' : 'типология отношений',
          'физикаобсуждение' : 'обсуждение физики', 'бизнесатематическая' : 'бизнес', 'аспектформулирование'  :'формулилование',
          'имуществаструктура' : 'структура имущества', 'капиталаструктура' : 'структура капитала', 'креативабренд' : 'креативный бренд',
          'investmentsvaluation' : 'investments valuation', 'discussionsocial' : 'social discussion', 'управлениясравнение' : 'сравнение уравление',
          'csdiscuss' : 'cs discuss', 'semanticspredicate' : 'semantics predicate', '1международное' : 'международное', 'излучателиплоские' : 'плоские излучатели',
          'мышлениеоперирование' : 'мышление, оперирование', 'проектамипонятие' : 'понятие проекта', 'памятиотличительные' : 'память',
          'сравнениеисточники' : 'источники сравнения', 'проектамиосновные' : '', 'otherclassification' : 'other classification',
          'проектаосновные' : 'основные проекты', 'visionfechner' : 'vision fechner', 'переходыфлуктуации' : 'переходы флуктуации',
          'modelsdynamic' : 'models dynamic', 'marketingsocial' : 'social marketing',  'развитияопределение' : 'определение развития',
          'медиарынокпонятие': 'понятие медиарынка', 'расчетоввизуализация' : 'визуализация расчетов', 'обеспеченияосновы' : 'основы обеспечения',
          'zeитvertreibтема' : 'zeitvertreib', 'системаходнофотонные' : 'фотон', 'персоналаработа' : 'работа персонала',
          'поискаусловие' : 'поиск условия', 'усилителиназначение' : 'назначение усилителей', 'компаниипонятие' : 'понятие компанияя',
          'телевиденияобработка' : 'обработка на телевидение', 'радиоуправлениятема' : 'радиоуправление','массивовалгоритм' : 'алгоритм с массивами',
          'программпараллельные' : 'параллельное программирование', 'сверхрешеткахспин' : 'решетка, спин', 
          'поведенияэтика' : 'этика поведения', 'направленийэтап' : '', 'managementhistory' : 'management history',
          'emergenceinternet' : 'emergence internet', 'methodscluster' : 'cluster methods', 'концентраторымост' : 'концентраторы, мост',
          'selectiondimension': 'dimention selection', 'treeslinear' : 'linear trees', 'analysismachine' : 'analysis machine',
          'rattleдерево' : 'rattle дерево', 'problemmarket' : 'market problem', 'controlprincipal' : 'control principal',
          'projectорганизация' : 'организация проекта', '2системы' : 'системы', 'свободылинейные' : '',
          'фильтровопределение' : 'определение фильтров', 'введениепредмет' : '', 'средсттехнико' : 'технические средства',
          'средствуправление' : 'средства управления', 'постнеклассической' : 'неклассический пост', 'разработкаистория' : 'разработка истории',
          'spssнабор' : 'spss набор', 'djangoнейросеть' : 'django нейросеть', 'структураопределение' : 'определение структуры',
          'исследованийчистые' : 'чистые исследования', 'modelsproject' : 'project models', 'lorawanособенность' : 'lorawan особенность',
          'уравнениевывод' : 'вывод уравнения', 'безопасностипонятие' : 'понятие безопасности', 'системыопределение' : 'определение системы',
          'классыобработка' : 'обработка классов', 'gamesequilibrium' : 'games equilibrium','компьютераосновные' : 'компьютер',
          'системмоделирование' : 'моделирование систем', 'анализасистемный' : 'системный анализ', 'courseintroduction' : '',
          'моделиметодология' : 'модели методологии', 'введениеосновная' : '', 'графовтеорем' : 'теоремы на графах', 
          'функциипроизводящие' : 'функции', 'карьереособенности' : 'особенности карьеры', 'введениевведение' : '', 
          'softwarecmt' : 'software cmt', 'внутрифирменное' : 'фирменное', 'сетипротоколы' : 'протоколы сети',
          'системинформационные' : 'информационные системы', 'statisticspopulation' : 'statistics population',
          'цепейэлектрическая' : 'электрическая цепь', 'systemsinformation' : 'information systems',
          'раннесредневековой' : 'средневековой', 'компьютераархитектура' : 'архитектура'}
            

In [139]:
data_2['Чистое содержание'] = data_2['Содержание курса'].apply(lambda x: x.lower())
data_2['Чистое содержание'] = data_2['Чистое содержание'].apply(lambda x : re.sub(r'\d+\.', '', x))
data_2['Чистое содержание'] = data_2['Чистое содержание'].apply(lambda x : re.sub(r'\d+\.\d+\.?', '', x))

In [230]:
data_2.head()

,SUBFIELDCODE,SUBFIELDNAME,DEGREE,SUBJECT,SEMESTER,START_YEAR,URL,Название курса,Содержание курса,Дата сет,Ключевые слова,key_words,many_words_keywords,cleaned_description,tf_idf_keywords,new_key_words,Чистое содержание
0,09.03.01,"Вычислительные машины, комплексы, системы и сети",Академический бакалавр,Математика,1,2014,https://www.hse.ru/edu/courses/292700221,Математика,Элементы линейной алгебрыТема 1.1. линейные пр...,hse,,"вероятность, распределение, экстремум, сложной...","['система линейный уравнение', 'вектор линейны...",элемент линейный алгебрытема линейный простран...,"{'функция': 0.325, 'случайный': 0.233, 'переме...",случайная величина; обобщение на функции; лини...,элементы линейной алгебрытема линейные простр...
1,09.03.01,"Вычислительные машины, комплексы, системы и сети",Академический бакалавр,Математика,2,2014,https://www.hse.ru/edu/courses/292700221,Математика,Элементы линейной алгебрыТема 1.1. линейные пр...,hse,,"вероятность, распределение, экстремум, сложной...","['система линейный уравнение', 'вектор линейны...",элемент линейный алгебрытема линейный простран...,"{'функция': 0.325, 'случайный': 0.233, 'переме...",случайная величина; обобщение на функции; лини...,элементы линейной алгебрытема линейные простр...
2,09.03.01,"Вычислительные машины, комплексы, системы и сети",Академический бакалавр,Физика,1,2014,https://www.hse.ru/edu/courses/292684119,Физика,Тема 1. Механика. Механические колебания и вол...,hse,,"импульс, поток, напряжённость, вектор, электро...","['волныкинематика колебание механический', 'ко...",механик механический колебание волныкинематика...,"{'колебание': 0.285, 'энергия': 0.256, 'вектор...",электромагнитные колебания и волныэ; гармониче...,тема механика. механические колебания и волны...
3,09.03.01,"Вычислительные машины, комплексы, системы и сети",Академический бакалавр,Физика,2,2014,https://www.hse.ru/edu/courses/292684119,Физика,Тема 1. Механика. Механические колебания и вол...,hse,,"импульс, поток, напряжённость, вектор, электро...","['волныкинематика колебание механический', 'ко...",механик механический колебание волныкинематика...,"{'колебание': 0.285, 'энергия': 0.256, 'вектор...",электромагнитные колебания и волныэ; гармониче...,тема механика. механические колебания и волны...
4,09.03.01,"Вычислительные машины, комплексы, системы и сети",Академический бакалавр,Основы права,1,2014,https://online.edu.ru/public/course.xhtml?face...,Основы права,Что такое право? Российское право на правовой...,itmo,,"семья, правонарушение, правового регулирования...","['гражданский собственность право', 'правовой ...",право российский право правовой карта мир взаи...,"{'право': 0.661, 'ответственность': 0.268, 'пр...",правового регулирования; правовой карте мира; ...,что такое право? российское право на правовой...


In [241]:
all_key_words = []

for course in tqdm(data_2.values):
    phrases = course[-6].split(', ')

    key_words = []
    
    for phrase in phrases:
        if len(phrase.split()) > 0:
            desc = course[-1]
            first = phrase.split()[0]
            last = phrase.split()[-1]
            

            if phrase != 'c++':
                first_indexes = [(m.start(0), m.end(0)) for m in re.finditer(first, desc)]
                last_indexes = [(m.start(0), m.end(0)) for m in re.finditer(last, desc)]

                min_index = np.inf

                for first_index in first_indexes:
                    for last_index in last_indexes:
                        dist = last_index[0] - first_index[0]
                        if dist >= 0 and dist < min_index:
                            min_index = dist
                            resulted_indicies = (first_index[0], last_index[1])

                d = desc[resulted_indicies[0] : resulted_indicies[1] + 1]
                d = re.sub(r'\d+\.', '', d)
                d = re.sub(r'\d+\.\d+\.?', '', d)
                d = d.split('.')
                d = list(set([word[0].upper() + word[1:] if len(word) != 0 else '' for word in d]))

                new_d = []
                for item in d:
                    if len(item.split()) < 5:
                        new_d.append(item)
                new_d = [item.strip() for item in new_d if len(item) > 0]

                d = '; '.join(new_d)

                for stop_word in handed_stop_words:
                    d = re.sub(stop_word, '', d.lower())

                for key, item in change.items():
                    d = re.sub(key, item, d)


                new_key_words = []
                for phrase in d.split('; '):
                    if len(phrase.split()) == 1:
                        if re.search(r'[А-Яа-я]+', phrase):
                            new_key_words.append(ru_lemmatizer.parse(phrase)[0].normal_form)
                        else:
                            new_key_words.append(en_lemmatizer.lemmatize(phrase))
                    else:
                        new_key_words.append(phrase)

                key_words.append('; '.join(new_key_words))
            else:
                key_words.append('c++')
    all_key_words.append('; '.join(list(set(key_words))))

100%|██████████| 53113/53113 [01:43<00:00, 512.27it/s]


In [242]:
data_2['new_key_words'] = all_key_words

In [250]:
del all_key_words

In [243]:
data_2.iloc[23]

SUBFIELDCODE                                                    09.03.02
SUBFIELDNAME           Автоматизация и управление в образовательных с...
DEGREE                                            Академический бакалавр
SUBJECT                                                 Иностранный язык
SEMESTER                                                               2
START_YEAR                                                          2014
URL                             https://www.hse.ru/edu/courses/301401516
Название курса                                          Иностранный язык
Содержание курса       Личность. Описание внешности, физических харак...
Дата сет                                                             hse
Ключевые слова                                                          
key_words              грамматика, коммуникативные humans living, язы...
many_words_keywords    ['внешность личность', 'living коммуникативный...
cleaned_description    личность внешность физически

In [244]:
data_2.iloc[23]['new_key_words']

'; социальные группы для обсуждения; грамматика:'

In [245]:
data_2.iloc[0]

SUBFIELDCODE                                                    09.03.01
SUBFIELDNAME            Вычислительные машины, комплексы, системы и сети
DEGREE                                            Академический бакалавр
SUBJECT                                                       Математика
SEMESTER                                                               1
START_YEAR                                                          2014
URL                             https://www.hse.ru/edu/courses/292700221
Название курса                                                Математика
Содержание курса       Элементы линейной алгебрыТема 1.1. линейные пр...
Дата сет                                                             hse
Ключевые слова                                                          
key_words              вероятность, распределение, экстремум, сложной...
many_words_keywords    ['система линейный уравнение', 'вектор линейны...
cleaned_description    элемент линейный алгебрытема

In [246]:
data_2.iloc[0]['new_key_words']

'  функция; переменный; случайная величина; линейный коэффициент корреляции; вероятность; линейных уравнений; сложной функции; распределение; линейных пространств; вектор; экстремум'

In [249]:
data_2.iloc[10]

SUBFIELDCODE                                                    09.03.01
SUBFIELDNAME            Вычислительные машины, комплексы, системы и сети
DEGREE                                            Академический бакалавр
SUBJECT                                              Физическая культура
SEMESTER                                                               2
START_YEAR                                                          2014
URL                    https://online.edu.ru/public/course.xhtml?face...
Название курса                                      Физическая культура 
Содержание курса        Тема 1. Общая концепция теории и методики физ...
Дата сет                                                            itmo
Ключевые слова         здоровый образ жизни студента, спорт, физиолог...
key_words              физическими упражнениями, физической культуры,...
many_words_keywords    ['упражнение физический', 'человек биохимическ...
cleaned_description    физический культура физическ

In [248]:
data_2.iloc[10]['new_key_words']

'; физическое воспитание,; здоровый образ жизни; физической культуры; центральная категория физической культуры; физическими упражнениями'

In [ ]:
data_2.to_csv('D')